In [8]:
import numpy as np
import pandas as pd
a = np.arange(9).reshape(3,3)
b = np.arange(3)
np.matmul(a,b)

array([ 5, 14, 23])

In [9]:
folder_path = "update_funk_500_on_500_each_300/"
ma = pd.read_csv(folder_path+'m_star1_a.csv',delimiter=';',header=None).to_numpy()[:,:-1]
ka = pd.read_csv(folder_path+'k_star1_a.csv',delimiter=';',header=None).to_numpy()[:,:-1]
mn = pd.read_csv(folder_path+'m_star1_n.csv',delimiter=';',header=None).to_numpy()[:,:-1]

m = np.linspace(0,np.sqrt(500),300)**2
k = np.linspace(0,np.sqrt(500),300)**2
m_bins = (m[1:]+m[:-1])/2
m_bins[1:] = m_bins[:-1]
m_bins[0] =0
k_bins = (k[1:]+k[:-1])/2
k_bins[1:] = k_bins[:-1]
k_bins[0] =0

def bin_(ka,k,bins):
    ka_bins = np.digitize(ka,bins)-1
    ka_bined = m[np.where(ka_bins<0,0,ka_bins)]
    ka_bined = np.where(ka_bins==k.shape[0]-1,-np.inf,ka_bined)
    return ka_bined, ka_bins#, np.where(ka_bins==k.shape[0]-1,False,True)
ka_bined, ka_bins = bin_(ka,k,k_bins)
mn_bined, ma_bins = bin_(mn,m,m_bins)
ma_bined, mn_bins = bin_(ma,m,m_bins)
a_bins = np.dstack((ma_bins,ka_bins))

In [ ]:
lam = 0.065
# from cupy.scipy.sparse import coo_array
from scipy.sparse import coo_array
# q_mat_a = np.zeros((k.shape[0]**2,k.shape[0]**2))
indices= np.zeros((k.shape[0]**2,2))
coordinates_old = []
coordinates_new = []

inverse = np.zeros((len(m),len(k),2,820))
inverse_count = np.zeros((len(m),len(k)),dtype=np.int32)
for i in range(len(k)):
    for j in range(len(m)):
#         inverse[a_bins[i,j,0],a_bins[i,j,1],:,inverse_count[a_bins[i,j,0],a_bins[i,j,1]]]=[i,j]
#         inverse_count[a_bins[i,j,0],a_bins[i,j,1]]+=1
# for i in range(len(k)):
#     for j in range(len(m)):
#         for l in range(inverse_count[i,j]):
            # coordinates_new.append(i+len(m)*j)
            # coordinates_old.append(inverse[i,j,0,l]+len(m)*inverse[i,j,1,l])
            coordinates_old.append(i+len(m)*j)
            coordinates_new.append(a_bins[i,j,0]+len(m)*a_bins[i,j,1])
data = np.ones(len(coordinates_new),dtype=np.float16)*lam

q_mat_a = coo_array((data,(coordinates_new,coordinates_old)),shape=(k.shape[0]**2,k.shape[0]**2),dtype=np.float64)            

        # indices[i+j*len(m)] = [i,j]
        # if ((i,j) in bins_2d_a):
        #     for l in range(len(m)):
        #         for n in range(len(k)):
        #             if np.all([i,j]==a_bins[l,n]):
        #                 coordinates.append([i+j*len(m),l+n*len(m)])
        #                 q_mat_a[i+j*len(m),l+n*len(m)]=lam
# np.max(inverse_count)

In [11]:
indices= np.zeros((k.shape[0]**2,2))
coordinates_old = []
coordinates_new = []

inverse = np.zeros((len(m),len(k),2,820))
inverse_count = np.zeros((len(m),len(k)),dtype=np.int32)
for i in range(len(k)):
    for j in range(len(m)):
#         inverse[mn_bins[i,j],j,:,inverse_count[mn_bins[i,j],j]]=[i,j]
#         inverse_count[mn_bins[i,j],j]+=1
# for i in range(len(k)):
#     for j in range(len(m)):
#         for l in range(inverse_count[i,j]):
            # coordinates_new.append(i+len(m)*j)
            # coordinates_old.append(inverse[i,j,0,l]+len(m)*inverse[i,j,1,l])
            coordinates_old.append(i+len(m)*j)
            coordinates_new.append(mn_bins[i,j]+len(m)*j)
data = np.ones(len(coordinates_new),dtype=np.float16)*(1-lam)
q_mat_n = coo_array((data,(np.array(coordinates_new),np.array(coordinates_old))),shape=(k.shape[0]**2,k.shape[0]**2),dtype=np.float64)     

In [12]:

q_mat = q_mat_a + q_mat_n
from scipy.sparse.linalg import eigs
eig_vals, eig_vecs = eigs(q_mat,k=4)
eig_vals

ArpackNoConvergence: ARPACK error -1: No convergence (900001 iterations, 1/4 eigenvectors converged)

In [ ]:
pdf_grid = np.zeros_like(ka)
for i in range(len(m)):
    for j in range(len(k)):
        pdf_grid[i,j] = eig_vecs[0,i+len(m)*j]

pdf_k = np.sum(pdf_grid,axis=0)
cdf_bcondk = np.matmul(np.cumsum(pdf_grid,axis=0),1/pdf_k)
import matplotlib.pyplot as plt